In [1]:
import time
from design import *
import importlib
import shutil
from utils import *
from openai import OpenAI
from prompts import *
import json
import numpy as np
from gymnasium.envs.robodesign.GPTCheetah import GPTCheetahEnv

In [2]:

folder_name = "results/CheetahEureka_morphology"
log_file = os.path.join(folder_name, "parameters.log")
logging.basicConfig(filename=log_file, level=logging.INFO, format="%(asctime)s - %(message)s")

# folder_name = setup_logging(div_flag=True)

best_fitness = float('-inf')  
best_morphology = None  
best_rewardfunc = None  
best_reward = None
best_material = None
best_efficiency = None

iterations = 5
morphology_nums = 16
rewardfunc_nums = 1

fitness_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
efficiency_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
fitness_list = []


In [ ]:
import prompts
class DGA:
    def __init__(self):
        api_key = "<api_key>"
        self.client = OpenAI(api_key=api_key)
        self.model = "gpt-4o-mini"


    def generate_morphology_eureka(self, morphology_nums, best_message, folder_name):

        messages = [
                    {"role": "system", "content": "You are a helpful mujoco robot designer"},
                    {"role": "user", "content": morphology_prompts + best_message + morphology_format}
                    ]
        
        responses = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            response_format={'type': 'json_object'},
            n=morphology_nums
        )

        for i, choice in enumerate(responses.choices):
            print(f"Response {i}:")
            print(json.dumps(choice.message.content, indent=4))

        parameter_list = [json.loads(choice.message.content).get('parameters', []) for choice in responses.choices]
        material_list = [compute_cheetah_volume(parameter) for parameter in parameter_list]

        xml_files = []
        for i, parameter in enumerate(parameter_list):
            if not isinstance(parameter, list):
                print(f"Skipping invalid parameter {i}: {parameter}")
                continue

            xml_file = cheetah_design(parameter)  
            filename = f"GPTCheetah_{i}.xml"
            file_path = os.path.join(folder_name, "assets", filename)
            xml_files.append(file_path)
            with open(file_path, "w") as fp:
                fp.write(xml_file)
            print(f"Successfully saved {filename}")
            
        return xml_files, material_list, parameter_list
    
        


In [5]:
eureka_morphology_prompts = """Role: You are a robot designer trying to design robot parameters to increase the fitness function as effective as possible.
Task: Your task is to design parameters of robot that will help agent achieve the fitness function as high as possible.
fintess function: walk distance/material cost.
Description: The HalfCheetah is a 2-dimensional robot consisting of 9 body parts and 8 joints connecting them (including two paws). The goal is to apply torque to the joints to make the cheetah run forward (right) as fast as possible, with a positive reward based on the distance moved forward and a negative reward for moving backward.
The cheetah's torso and head are fixed, and torque can only be applied to the other 6 joints over the front and back thighs (which connect to the torso), the shins (which connect to the thighs), and the feet (which connect to the shins).

  <mujoco model="cheetah">
    <compiler angle="radian" coordinate="local" inertiafromgeom="true" settotalmass="14"/>
    <default>
      <joint armature=".1" damping=".01" limited="true" solimplimit="0 .8 .03" solreflimit=".02 1" stiffness="8"/>
      <geom conaffinity="0" condim="3" contype="1" friction=".4 .1 .1" rgba="0.8 0.6 .4 1" solimp="0.0 0.8 0.01" solref="0.02 1"/>
      <motor ctrllimited="true" ctrlrange="-1 1"/>
    </default>
    <size nstack="300000" nuser_geom="1"/>
    <option gravity="0 0 -9.81" timestep="0.01"/>
    <asset>
      <texture builtin="gradient" height="100" rgb1="1 1 1" rgb2="0 0 0" type="skybox" width="100"/>
      <texture builtin="flat" height="1278" mark="cross" markrgb="1 1 1" name="texgeom" random="0.01" rgb1="0.8 0.6 0.4" rgb2="0.8 0.6 0.4" type="cube" width="127"/>
      <texture builtin="checker" height="100" name="texplane" rgb1="0 0 0" rgb2="0.8 0.8 0.8" type="2d" width="100"/>
      <material name="MatPlane" reflectance="0.5" shininess="1" specular="1" texrepeat="60 60" texture="texplane"/>
      <material name="geom" texture="texgeom" texuniform="true"/>
    </asset>
    <worldbody>
    <light cutoff="100" diffuse="1 1 1" dir="-0 0 -1.3" directional="true" exponent="1" pos="0 0 1.3" specular=".1 .1 .1"/>
    <geom conaffinity="1" condim="3" material="MatPlane" name="floor" pos="0 0 0" rgba="0.8 0.9 0.8 1" size="40 40 40" type="plane"/>
    <body name="torso" pos="0 0 {height}">
      <joint armature="0" axis="1 0 0" damping="0" limited="false" name="ignorex" pos="0 0 0" stiffness="0" type="slide"/>
      <joint armature="0" axis="0 0 1" damping="0" limited="false" name="ignorez" pos="0 0 0" stiffness="0" type="slide"/>
      <joint armature="0" axis="0 1 0" damping="0" limited="false" name="ignorey" pos="0 0 0" stiffness="0" type="hinge"/>
      <geom fromto="{param1} 0 0 {param2} 0 0" name="torso" size="{param17}" type="capsule"/>
      <geom fromto="{param2} 0 0 {param3} 0 {param4}" name="head"  size="{param18}" type="capsule"/>

      <body name="bthigh" pos="{param1} 0 0">
        <joint axis="0 1 0" damping="6" name="bthigh" pos="0 0 0" range="-.52 1.05" stiffness="240" type="hinge"/>
        <geom fromto = "0 0 0 {param5} 0 {param6}" name="bthigh" size="{param19}" type="capsule"/>
        <body name="bshin" pos="{param5} 0 {param6}">
          <joint axis="0 1 0" damping="4.5" name="bshin" pos="0 0 0" range="-.785 .785" stiffness="180" type="hinge"/>
          <geom fromto = "0 0 0 {param7} 0 {param8}" name="bshin" rgba="0.9 0.6 0.6 1" size="{param20}" type="capsule"/>
          <body name="bfoot" pos="{param7} 0 {param8}">
            <joint axis="0 1 0" damping="3" name="bfoot" pos="0 0 0" range="-.4 .785" stiffness="120" type="hinge"/>
            <geom fromto = "0 0 0 {param9} 0 {param10}" name="bfoot" rgba="0.9 0.6 0.6 1" size="{param21}" type="capsule"/>
          </body>
        </body>
      </body>

      <body name="fthigh" pos="{param2} 0 0">
        <joint axis="0 1 0" damping="4.5" name="fthigh" pos="0 0 0" range="-1.5 0.8" stiffness="180" type="hinge"/>
        <geom fromto = "0 0 0 {param11} 0 {param12}" name="fthigh" size="{param22}" type="capsule"/>
        <body name="fshin" pos = "{param11} 0 {param12}">
          <joint axis="0 1 0" damping="3" name="fshin" pos="0 0 0" range="-1.2 1.1" stiffness="120" type="hinge"/>
          <geom fromto = "0 0 0 {param13} 0 {param14}" rgba="0.9 0.6 0.6 1" size="{param23}" type="capsule"/>
          <body name="ffoot" pos="{param13} 0 {param14}">
            <joint axis="0 1 0" damping="1.5" name="ffoot" pos="0 0 0" range="-3.1 -0.3" stiffness="60" type="hinge"/>
            <geom fromto = "0 0 0 {param15} 0 {param16}" name="ffoot" rgba="0.9 0.6 0.6 1" size="{param24}" type="capsule"/>
          </body>
        </body>
      </body>
    </body>

    </worldbody>
    <actuator>
    <motor gear="120" joint="bthigh" name="bthigh"/>
    <motor gear="90" joint="bshin" name="bshin"/>
    <motor gear="60" joint="bfoot" name="bfoot"/>
    <motor gear="120" joint="fthigh" name="fthigh"/>
    <motor gear="60" joint="fshin" name="fshin"/>
    <motor gear="30" joint="ffoot" name="ffoot"/>
    </actuator>
    </mujoco>

There are also some design parameters and their fitness. 
Please carefully observe these parameters and their fitness, try to design a new parameter to further improve the fitness.

"""

In [ ]:
designer = DGA()
rewardfunc_list = [f'results/CheetahEureka_morphology/env/GPTrewardfunc_{i}.py' for i in range(0,1)]
best_message = ""
for iter in range(iterations):
    morphology_list, material_list, parameter_list = designer.generate_morphology_eureka(morphology_nums, best_message, folder_name)
    for i, rewardfunc in enumerate(rewardfunc_list):
        for j, morphology in enumerate(morphology_list):

            print(i, rewardfunc)
            print(j, morphology)
            shutil.copy(morphology, "GPTCheetah.xml")
            shutil.copy(rewardfunc, "GPTrewardfunc.py")         

            import GPTrewardfunc
            importlib.reload(GPTrewardfunc)  # 重新加载模块
            from GPTrewardfunc import _get_rew
            GPTCheetahEnv._get_rew = _get_rew

            env_name = "GPTCheetahEnv"
            model_path = Train(j,  i, folder_name, total_timesteps=5e5)
            fitness, reward = Eva(model_path)

            material = material_list[j]
            efficiency = fitness/material
            fitness_matrix[i][j] = fitness
            efficiency_matrix[i][j] = efficiency
            
            logging.info("___________________finish coarse optimization_____________________")
            logging.info(f"morphology: {j}, rewardfunc: {i}, material cost: {material} reward: {reward} fitness: {fitness} efficiency: {efficiency}")

    best_index = np.argmax(efficiency_matrix)
    best_efficiency = np.max(efficiency_matrix[:][0])
    best_parameter = parameter_list[best_index]
    best_message = eureka_morphology_prompts + f"\n best parameter:{best_parameter} \n" + f"best fintess:{best_efficiency}" 

In [11]:
best_efficiency = 8384.79109895467

8384.79109895467

In [12]:
best_index = 15

15

In [14]:
best_parameter = [-0.63,
 0.61,
 0.94,
 0.51,
 0.41,
 -0.71,
 0.65,
 -1.1,
 0.81,
 -1.5,
 -0.56,
 -0.6,
 -0.68,
 -1.2,
 0.53,
 -1.6,
 0.029,
 0.02,
 0.018,
 0.013,
 0.012,
 0.018,
 0.012,
 0.012]

[-0.63,
 0.61,
 0.94,
 0.51,
 0.41,
 -0.71,
 0.65,
 -1.1,
 0.81,
 -1.5,
 -0.56,
 -0.6,
 -0.68,
 -1.2,
 0.53,
 -1.6,
 0.029,
 0.02,
 0.018,
 0.013,
 0.012,
 0.018,
 0.012,
 0.012]